### Malaria Detection using Keras

In [1]:
import numpy as np 
import pandas as pd

import tensorflow as tf
import cv2
import tensorflow.keras
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Input, BatchNormalization, ZeroPadding2D, Dropout
from tensorflow.keras.models import Sequential, Model

### Dataset Exploration

In [2]:
data_dir="cell_images/"
batch_size = 32
img_height = 180
img_width = 180
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 27558 files belonging to 2 classes.
Using 22047 files for training.


In [3]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['Parasitized', 'Uninfected']


In [5]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

### Understanding our model
- The model we have created is for understanding of all the concepts one would require for understanding and building your deep learning models. The model we have created here uses functional model structure defined in Keras.

In [6]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [8]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [10]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("malariadetection.h5")


Epoch 1/3
689/689 [==============================] - 1466s 2s/step - loss: 0.5517 - accuracy: 0.7200 - val_loss: 0.2211 - val_accuracy: 0.9193
Epoch 2/3
689/689 [==============================] - 564s 819ms/step - loss: 0.1875 - accuracy: 0.9360 - val_loss: 0.1900 - val_accuracy: 0.9421
Epoch 3/3
689/689 [==============================] - 569s 825ms/step - loss: 0.1502 - accuracy: 0.9520 - val_loss: 0.1520 - val_accuracy: 0.9532


In [11]:

# It can be used to reconstruct the model identically.
reconstructed_model = tensorflow.keras.models.load_model("malariadetection.h5")

In [16]:
def image_reader(path):
    t=cv2.imread(path)
    t=cv2.cvtColor(t, cv2.COLOR_BGR2RGB)
    t=cv2.resize(t, (180,180))

    return t
evaluation=[]
evaluation.append(image_reader(r"cell_images\C33P1thinF_IMG_20150619_114756a_cell_179.png"))
evaluation=np.asarray(evaluation)
print(evaluation.shape)
res=model.predict(evaluation)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
training_labels=le.fit_transform(class_names)
target=le.inverse_transform(res.argmax(1))
print(target)

(1, 180, 180, 3)
['Parasitized']
